# TOAR REST-API Demo

In [757]:
from io import StringIO
import json
import pandas as pd
import requests
import time
from pathlib import Path
import zipfile
import io
import os

# Configuration
Insert your access token that can be created in your user profile. A registered user is required for this step.
<br>***Note!** Your access token will be only valid for one hour after creation.*

In [698]:
accessToken = 'eyJraWQiOiI1MTQ0NTM4ODMwNDI3Mjk1NzM2MjI2Mjc4MDM1NTUxMzg3NzY1NDMxNTY4OTIyMjAiLCJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIzNTU0ZTcyYS02MjUzLTQ2YTktOWUyZS1lMWUzNGM3MGNkNTUiLCJhdWQiOiJ0b2FyX3NlcnZpY2VzIiwic2NvcGUiOiJkaXNwbGF5X25hbWUgb2ZmbGluZV9hY2Nlc3MgZW1haWwgcHJvZmlsZSBlZHVwZXJzb25fc2NvcGVkX2FmZmlsaWF0aW9uIGVkdXBlcnNvbl9lbnRpdGxlbWVudCBlZHVwZXJzb25fdW5pcXVlX2lkIGVkdXBlcnNvbl9hc3N1cmFuY2UiLCJhdXRoX3RpbWUiOjE3NjQ3MTk2MjgsImlzcyI6Imh0dHBzOi8vbG9naW4uaGVsbWhvbHR6LmRlL29hdXRoMiIsImV4cCI6MTc2NDcyMzY0NiwiaWF0IjoxNzY0NzE5NjQ2LCJqdGkiOiJiNmM3NWQzNC1hMzFhLTRhNjYtOWViYS0wYTBiMmVmYzYyNjgiLCJjbGllbnRfaWQiOiJ0b2FyX3NlcnZpY2VzIn0.ChbIqoatGdvppIV_CuMvfDw_EPYuXnqKOw4LXGaN4S0yUbDCxvX6p-KSaYXUDVvGIVYqIIa-gWb3jqDcoXa4bd2ilFCheK4pz2LUNpgUul6WS6wvn8Omelex39Gahf_E9EK40__EL-7i24SnI3jYqRhqYjtvDI_sjWFZDLOmokdwaPjKOELRVHujkPiZIGSmeadV29FpQyHWbbxooC5EFlmRNkFOc-Y92pKBGhEknD_dU8nO-mj3L7KYMXV6W93IY08QzsMNXgsDQtWE4JnihAMlDZegW-pXbhQRkFTQvqIxClG21RTC37o7Ni0Dt9UWOsECSzcDBRa24jKWmj5aBQHHpwc3m4yi0YTRaNUbmHD0YvQmYCIVUiIyOh-4O927mw9jVJYpIjMiURp-J2RGVqzXOQQJBdPRBhDP8hfwG1f-M5H-SIYxU50AVMdzg6NuWvomJiEKoSX_w3Zk3gCn1Phuy51AsAqSHkOEU7pEO-Z4FfuQ-vKiX5VKUaJDjblCpX4F905-05XjvspX7fm7CcJ_sZuMwVK7-lekofUzt9vndE0TI19bVIg9UTMhitT1EiFTxvR9LcyYsIfM-C1XFpqDSfm_5-yK7Bgq5lEPFK_syk7j1pb7pn_-AWP0r0XsNTON4yYfnjW5niwMQvcAjb1klYgq9BgwXdfj0B9yiv0'

In [700]:
TOAR_SERVICE_URL="https://toar-data.fz-juelich.de/api/v2/"
headers = {'AccessToken': accessToken}
TIME_SERIES_ID = 21
flags = None

In [230]:
file_name = f"toar_time_series_{TIME_SERIES_ID}.csv"
download_folder = Path("data_downloads")
download_folder.mkdir(exist_ok = True)
output_path = download_folder / file_name

# Sample Code from TOAR

In [117]:
# get data from the REST service
flags = None
timeseries_id=21
if flags:
    result=requests.get(TOAR_SERVICE_URL+f'data/timeseries/{timeseries_id}?flags={flags}&format=csv', headers=headers)
else:
    result=requests.get(TOAR_SERVICE_URL+f'data/timeseries/{timeseries_id}?format=csv', headers=headers)
timeseries_meta = json.loads("\n".join([line[1:] for line in result.text.split('\n') if line.startswith('#')]))
df = pd.read_csv(StringIO(result.text), comment='#', header=1, sep=',',names=["time","value","flags","version","timeseries_id"],parse_dates=["time"],index_col="time",date_parser=lambda x:pd.to_datetime(x,utc=True),infer_datetime_format=True)

# print, what I got
print(f"metadata: {timeseries_meta}")
print(f"data: {df}")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [119]:
# example of how to access metadata
stationmeta=timeseries_meta["station"]
variable=timeseries_meta["variable"]
data_start_date = min(df.index).strftime("%Y-%m-%d %H:%M:%S+00")
data_end_date = min(df.index).strftime("%Y-%m-%d %H:%M:%S+00")

# show stationmeta data
print(f'retrieved data from station {stationmeta["name"]} for variable {variable["name"]} with timerange {data_start_date} - {data_end_date}')
print(f'station coordinates: {stationmeta["coordinates"]}')

retrieved data from station DUCKWORTH & ORDINANCE for variable o3 with timerange 2012-01-01 04:30:00+00 - 2012-01-01 04:30:00+00
station coordinates: {'lat': 47.56806, 'lng': -52.70222, 'alt': 7.0}


# Refined Download Script

In [232]:
print(f"Begin with download of time series {TIME_SERIES_ID} ...")

start_time = time.time()

response = requests.get(
    f"{TOAR_SERVICE_URL}data/timeseries/{TIME_SERIES_ID}?format=csv",
    headers = headers,
    timeout=(3.05, 5)
)

end_time = time.time()
elapsed_time = end_time - start_time

# Check that your access token is valid
if response.status_code in(401, 403):
    raise RuntimeError(
        "Access token is missing, invalid, or has expired."
    )

# Check that the given URL and timeseries_id is valid
if response.status_code == 404:
    raise RuntimeError(
        "Requested resource does not exist. Please check for wrong URL or invalid timeseries_id in the configuration above."
    )
    
# Checks that the TOAR database is available
if response.status_code == 503:
    raise RuntimeError(
        "Service is not avaiable. TOAR database may be temporarily offline or overloaded."
    )

print(f"Download finished in {elapsed_time:.2f} seconds.")

# Save the downloaded data on your local drive
print(f"Create folder with name {output_path} and a download file {file_name}.")

with open(output_path, "w", encoding="utf-8") as f:
    f.write(response.text)

print("Done.")


Begin with download of time series 21 ...
Download finished in 0.73 seconds.
Create folder with name data_downloads\toar_time_series_21.csv and a download file toar_time_series_21.csv.
Done.


# Explore data access and endpoint search

In [292]:
# provides all stations from Cologne, Germany(DE)
cologne = requests.get("https://toar-data.fz-juelich.de/api/v2/stationmeta/?country=DE&state=Nordrhein-Westfalen&name=Köln&limit=None", timeout=(3.05, 5))
stations_cologne = cologne.json()
print(stations_cologne)

[{'id': 37, 'codes': ['DENW053'], 'name': 'Köln-Chorweiler', 'coordinates': {'lat': 51.019345, 'lng': 6.884636, 'alt': 45.0}, 'coordinate_validation_status': 'not checked', 'country': 'Germany', 'state': 'Nordrhein-Westfalen', 'type': 'background', 'type_of_area': 'urban', 'timezone': 'Europe/Berlin', 'additional_metadata': {'rice_production': 0.0, 'station_alt_flag': '0', 'wheat_production': 0.0, 'google_resolution': '153', 'soybean_production': 0.0, 'station_google_alt': '46', 'station_reported_alt': '45', 'station_landcover_description': 'Urbanandbuilt-up:53.9%,Croplands:20.3%,Mixedforest:17.4%,Cropland/Naturalvegetationmosaic:5.7%,EvergreenNeedleleafforest:2.1%', 'station_max_population_density_5km': '31605'}, 'aux_images': [], 'aux_docs': [], 'aux_urls': [], 'globalmeta': {'mean_topography_srtm_alt_90m_year1994': 46.0, 'mean_topography_srtm_alt_1km_year1994': 46.59005145797599, 'max_topography_srtm_relative_alt_5km_year1994': 19.0, 'min_topography_srtm_relative_alt_5km_year1994': 

In [304]:
# all timeseries from station 404 with data origin 'instrument' 
resp = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/timeseries/"
    "?station_id=404"
    "&data_origin=Instrument"
    "&variable_id=13",
    timeout=(3.05, 10)
)
ts = resp.json()
print(ts)

[{'id': 19987, 'label': '', 'order': 1, 'sampling_frequency': 'hourly', 'aggregation': 'mean of two values', 'data_start_date': '2012-01-01T00:00:00+00:00', 'data_end_date': '2025-12-02T17:00:00+00:00', 'data_origin': 'instrument', 'data_origin_type': 'measurement', 'provider_version': 'N/A', 'sampling_height': 2.0, 'additional_metadata': {}, 'doi': '', 'coverage': -1.0, 'station': {'id': 404, 'codes': ['DENW212'], 'name': 'Köln Turiner Straße', 'coordinates': {'lat': 50.947367, 'lng': 6.957701, 'alt': 49.0}, 'coordinate_validation_status': 'not checked', 'country': 'Germany', 'state': 'Nordrhein-Westfalen', 'type': 'traffic', 'type_of_area': 'urban', 'timezone': 'Europe/Berlin', 'additional_metadata': {'rice_production': 0.0, 'station_alt_flag': '0', 'wheat_production': 0.0, 'google_resolution': '153', 'soybean_production': 0.0, 'station_google_alt': '55', 'station_reported_alt': '49', 'station_landcover_description': 'Urbanandbuilt-up:48.3%,Mixedforest:23.8%,Croplands:17.1%,Cropland/

# Data download via analysis service

In [743]:
# Download the data on a specific variable from a specific station in a specific date range
resp = requests.get( 
    "https://toar-data.fz-juelich.de/api/v2/analysis/data/timeseries/?station_id=404&limit=3&variable_id=13&daterange=2010-01-01T00:00:00,2024-12-31T23:59:59", 
    headers=headers,
    timeout=(3.05, 20) 
)

week_data = resp.json() 
print(week_data)

{'task_id': '4f0719bb-7c44-45ce-abc0-8c791202b941', 'status': 'https://toar-analysis.fz-juelich.de/api/v2/status/4f0719bb-7c44-45ce-abc0-8c791202b941'}


In [745]:
result_url = "https://toar-analysis.fz-juelich.de/api/v2/status/4f0719bb-7c44-45ce-abc0-8c791202b941"
resp = requests.get(result_url)
with open("toar_result.zip", "wb") as f:
    f.write(resp.content)

print("Data downloaded: toar_result.zip")

Data downloaded: toar_result.zip


In [747]:
all_data = []

with zipfile.ZipFile("toar_result.zip", "r") as z:
    json_files = [n for n in z.namelist() if n.endswith(".json")]
    print("Gefundene JSON-Dateien:", json_files)
    
    for file_name in json_files:
        with z.open(file_name) as f:
            #  Decode
            content = f.read().decode("utf-8").strip()
            if not content:
                print(f"File {file_name} is empty, skip.")
                continue

            # Skip the beginning of the Json file (otherwise you will get an eror)
            lines = content.splitlines()
            json_text = "\n".join([line for line in lines if not line.startswith("#")])
            
            try:
                obj = json.loads(json_text)
                
                # Extract data on variable
                if isinstance(obj, dict) and "data" in obj:
                    all_data.extend(obj["data"])
                elif isinstance(obj, list):
                    all_data.extend(obj)
                else:
                    print(f"File {file_name} does not inherit data array.")
                    
            except json.JSONDecodeError:
                print(f"File {file_name} could not be parsed.")

# Create dataframe
df = pd.DataFrame(all_data)
if not df.empty and 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])

print(df.head())
print("Number of rows:", len(df))

Gefundene JSON-Dateien: ['19987.json', '31544.json', '31887.json']
                    datetime    value  flags                       version  \
0  2012-01-01 00:00:00+00:00  746.203      1  000001.000000.00000000000000   
1  2012-01-01 01:00:00+00:00  176.818      1  000001.000000.00000000000000   
2  2012-01-01 02:00:00+00:00   80.217      1  000001.000000.00000000000000   
3  2012-01-01 03:00:00+00:00   64.593      1  000001.000000.00000000000000   
4  2012-01-01 04:00:00+00:00   48.118      1  000001.000000.00000000000000   

   timeseries_id  
0          19987  
1          19987  
2          19987  
3          19987  
4          19987  
Number of rows: 86568


In [749]:
with zipfile.ZipFile("toar_result.zip", "r") as z:
    for file_name in z.namelist():
        with z.open(file_name) as f:
            content = f.read()
            print(file_name, content[:500])

19987.json b'# This file was generated from the request \'https://toar-data.fz-juelich.de/api/v2/data/timeseries/?station_id=404&limit=3&variable_id=13&daterange=2010-01-01T00:00:00,2024-12-31T23:59:59\' received at 2025-12-03 00:29:05.820894+00:00\n{"metadata": {"id": 19987, "label": "", "order": 1, "sampling_frequency": "hourly", "aggregation": "mean of two values", "data_start_date": "2012-01-01T00:00:00+00:00", "data_end_date": "2025-12-02T21:00:00+00:00", "data_origin": "instrument", "data_origin_type": "m'
31544.json b'# This file was generated from the request \'https://toar-data.fz-juelich.de/api/v2/data/timeseries/?station_id=404&limit=3&variable_id=13&daterange=2010-01-01T00:00:00,2024-12-31T23:59:59\' received at 2025-12-03 00:29:05.820894+00:00\n{"metadata": {"id": 31544, "label": "SPP.DE_DENW212_PM1_automatic_nephelometry_beta_Duration-1hour", "order": 2, "sampling_frequency": "hourly", "aggregation": "none", "data_start_date": "2013-12-31T23:00:00+00:00", "data_end_date":

## Via time series endpoint and as csv

In [754]:
# all timeseries from station 404 with data origin 'instrument' 
resp = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/timeseries/"
    "?station_id=404"
    "&data_origin=Instrument"
    "&variable_id=13",
    headers=headers,
    timeout=(3.05, 10)
)
ts = resp.json()
print(ts)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# single timeseries from station 404 
result = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/data/timeseries/19987?format=csv",
    headers=headers,
    timeout=(3.05, 10)
)

df = pd.read_csv(StringIO(result.text), comment='#', header=1, sep=',',names=["time","value","flags","version","timeseries_id"],parse_dates=["time"],index_col="time",date_parser=lambda x:pd.to_datetime(x,utc=True),infer_datetime_format=True)
# print results from download
print(f"metadata: {timeseries_meta}")
print(f"data: {df}")

In [ ]:
# Filter a wanted date range
start_date = "2024-11-01"
end_date   = "2024-11-07"
df_filtered = df.loc[start_date:end_date]

# Locally save csv file
df_filtered.to_csv("filtered_timeseries.csv", encoding="utf-8")

print("Saved filtered data. Number of rows:", len(df_filtered))
print(df_filtered.head())

#### Date arithmetic

In [ ]:
# Load csv into dataframe
df = pd.read_csv(
    StringIO(result.text),
    comment='#',
    header=0,
    names=["time","value","flags","version","timeseries_id"],
    parse_dates=["time"],
    index_col="time"
)

df.index = df.index.tz_convert('UTC') if df.index.tzinfo else df.index.tz_localize('UTC')

# define range
start_date = pd.Timestamp("2024-11-01", tz="UTC")
end_date   = pd.Timestamp("2024-11-07", tz="UTC")

# Create a folder to store the csv files
output_dir = "timeseries_csvs"
os.makedirs(output_dir, exist_ok=True)

#  Iterate over each day in time range and save each as a csv file
current_day = start_date
while current_day <= end_date:
    next_day = current_day + timedelta(days=1)
    
    # Data for one day
    mask = (df.index >= current_day) & (df.index < next_day)
    daily_data = df.loc[mask]
    
    if not daily_data.empty:
        filename = f"timeseries_{current_day.date()}.csv"
        filepath = os.path.join(output_dir, filename)
        daily_data.to_csv(filepath, encoding="utf-8")
        print(f"{filename} gespeichert. Zeilen: {len(daily_data)}")
    
    current_day = next_day